In [ ]:
import sys
sys.path.append('..')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from loader import BoschTechniquesDataset, TramDataset
from transformers import BertTokenizer

In [ ]:
df = pd.concat([pd.read_json("../datasets/tram_train.json"),  pd.read_json("../datasets/tram_test.json")], ignore_index=True)
df


In [ ]:
with pd.option_context('display.max_rows', None, 
                       'display.max_columns', None, 
                       'display.max_colwidth', None):  # Set max_colwidth to None
    filtered_df = df[df['labels'].apply(lambda x: 'T1112' in x or 'T1547.001' in x)]
    filtered_df = filtered_df[filtered_df['sentence'].apply(lambda x: 'attack' in x)]
    display(filtered_df[['sentence', 'labels']])

In [ ]:
TAS0= [x for x in dftest.labels.explode().unique().tolist() if str(x).startswith("T") and not (str(x).startswith("TA"))]
len(TAS0)

In [ ]:
TAS1= [x for x in df.labels.explode().unique().tolist() if str(x).startswith("T") and not (str(x).startswith("TA"))]
len(TAS1)

In [ ]:
len(set(TAS0).difference(TAS1))

In [ ]:
len(set(TAS0).intersection(TAS1))

In [ ]:
stop, "Choose reference dataset"

option 1:

In [ ]:
# this is just because the Dataset class needs a tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
df_train = pd.read_json("../datasets/bosch_train.json")
df_test = pd.read_json("../datasets/bosch_test.json")
dataset_train = BoschTechniquesDataset(df_train, tokenizer)
dataset_test = BoschTechniquesDataset(df_test, tokenizer)

option 2:

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
df_train = pd.read_json("../datasets/tram_train.json")
df_test = pd.read_json("../datasets/tram_test.json")
dataset_train = TramDataset(df_train, tokenizer)
dataset_test = TramDataset(df_test, tokenizer)

In [ ]:
labels = []
for item in dataset_train:
    labels.append(item['labels'].cpu().numpy())
for item in dataset_test:
    labels.append(item['labels'].cpu().numpy())
labels = np.array(labels)
labels

In [ ]:
l_cumsum = labels.sum(axis=0)
l_cumsum, l_cumsum.shape

In [ ]:
l_cumsum = labels.sum(axis=0)
q05 = np.quantile(l_cumsum, 0.5)
mean = np.mean(l_cumsum)
q05, mean

In [ ]:
l_cumsum

In [ ]:
l_above_mean = np.where(l_cumsum > mean, l_cumsum, 0)
l_above_mean

In [ ]:
l_above_median = np.where(l_cumsum > q05, l_cumsum, 0)
l_above_median

In [ ]:
vtop_10 = sorted(l_cumsum, reverse=True)[9]
l_top10 = np.where(l_cumsum >= vtop_10, l_cumsum, 0)
l_top10

In [ ]:
len(np.delete(l_above_median, np.where(l_above_median == 0)))

In [ ]:
len(np.delete(l_above_mean, np.where(l_above_mean == 0)))

In [ ]:
len(np.delete(l_top10, np.where(l_top10 == 0)))

In [ ]:
stop, "CHOOSE DATASET!"

In [ ]:
from const import BOSCH_TECHNIQUES_LABELS, TRAM_TECHNIQUES_LABELS
labels = TRAM_TECHNIQUES_LABELS

In [ ]:
id2label = {i: l for i, l in enumerate(labels)}
label_mean = [id2label[x] for x in np.nonzero(l_above_mean)[0]]
label_mean

In [ ]:
label_median = [id2label[x] for x in np.nonzero(l_above_median)[0]]
label_median

In [ ]:
label_top10 = [id2label[x] for x in np.nonzero(l_top10)[0]]
label_top10